In [3]:
# 간선 리스트 (edge_index)
edges = [(1, 2), (2, 0)]  # 방향 그래프
num_nodes = 3

# adjacency matrix 초기화
adj = [[0 for _ in range(num_nodes)] for _ in range(num_nodes)]

# 간선 정보 반영
for src, dst in edges:
    adj[src][dst] = 1
    adj[dst][src] = 1

# 출력
print("Adjacency Matrix (list):")
for row in adj:
    print(row)


Adjacency Matrix (list):
[0, 0, 1]
[0, 0, 1]
[1, 1, 0]


In [4]:
import numpy as np

edges = [(0, 1), (1, 2), (2, 0)]
num_nodes = 3

adj = np.zeros((num_nodes, num_nodes), dtype=int)

for src, dst in edges:
    adj[src][dst] = 1
    adj[dst][src] = 1

print("Adjacency Matrix (NumPy):")
print(adj)


Adjacency Matrix (NumPy):
[[0 1 1]
 [1 0 1]
 [1 1 0]]


In [5]:
import torch

edges = [(0, 1), (1, 2), (2, 0)]
num_nodes = 3

adj = torch.zeros((num_nodes, num_nodes), dtype=torch.float)

for src, dst in edges:
    adj[src][dst] = 1.0
    adj[dst][src] = 1.0

print("Adjacency Matrix (Torch):")
print(adj)

Adjacency Matrix (Torch):
tensor([[0., 1., 1.],
        [1., 0., 1.],
        [1., 1., 0.]])


In [6]:
from scipy import sparse
from torch_geometric.utils import to_scipy_sparse_matrix

C:\Users\ukg_1\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root="Cora",name="Cora")
data = dataset[0]
print(data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


Processing...
Done!


In [10]:
print("Edge Index:")
print(data.edge_index)
print("Node Features:")
print(data.x)

Edge Index:
tensor([[ 633, 1862, 2582,  ...,  598, 1473, 2706],
        [   0,    0,    0,  ..., 2707, 2707, 2707]])
Node Features:
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [11]:
def get_adjacency_matrix(edge_index):
    # working with scipy sparse since current PyTorch version doesn't support sparse x sparse multiplication
    adj = to_scipy_sparse_matrix(edge_index)
    adj += sparse.eye(adj.shape[0])  # add self loops
    degree_for_norm = sparse.diags(np.power(np.array(adj.sum(1)), -0.5).flatten())  # D^(-0.5)
    adj_hat_csr = degree_for_norm.dot(adj.dot(degree_for_norm))  # D^(-0.5) * A * D^(-0.5)
    adj_hat_coo = adj_hat_csr.tocoo().astype(np.float32)
    # to torch sparse matrix
    indices = torch.from_numpy(np.vstack((adj_hat_coo.row, adj_hat_coo.col)).astype(np.int64))
    values = torch.from_numpy(adj_hat_coo.data)
    adjacency_matrix = torch.sparse_coo_tensor(indices, values, torch.Size(adj_hat_coo.shape))

    return adjacency_matrix, adj_hat_csr

In [14]:
matrix, _ = get_adjacency_matrix(data.edge_index)
print(matrix)

tensor(indices=tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
                       [   0,  633, 1862,  ..., 1473, 2706, 2707]]),
       values=tensor([0.2500, 0.2500, 0.2236,  ..., 0.2000, 0.2000, 0.2000]),
       size=(2708, 2708), nnz=13264, layout=torch.sparse_coo)
